# Install required packages

In [0]:
!pip install --upgrade "ibm-watson>=3.2.0"

from ibm_watson import PersonalityInsightsV3

import json
from ibm_watson import ApiException
from google.colab import files
from google.colab import drive

# Initialize Watson API

In [0]:
personality_insights = PersonalityInsightsV3(
    version='2017-10-13',
    iam_apikey='{API_KEY}',
    url='https://gateway.watsonplatform.net/personality-insights/api'
)

# Get the result from Watson API

In [0]:
try:
  valuesT1 = []
  valuesT2 = []
  with open("all.txt") as json_data:
     data = json.load(json_data)
  
  for user in data["t1"]:
    response = personality_insights.profile(content=user, accept="application/json", content_type = 'application/json' )
    result = response.get_result()
    valuesT1.append(result['values'])
    
  for user in data["t2"]:
    response = personality_insights.profile(content=user, accept="application/json", content_type = 'application/json' )
    result = response.get_result()
    valuesT2.append(result['values'])
  
except ApiException as ex:
  print("Method failed with status code " + str(ex.code) + ": " + ex.message)

# Function who returns the item fron an array

In [0]:
def getPercentile(arr, key):
  for item in arr:
    if item["name"] == key:
      return item
  

# Calculate BCM for t1

In [0]:
BHV_M1 = valuesT1[0]
BHV_M2 = valuesT2[0]

ego = 0.2
thrashold = 0.05
final_result_t1 = []
index = 0

for j in valuesT1[1:4]:
  BHV_M1_temp = {}
  for value in j:
    BHV_M1_score = getPercentile(BHV_M1, value["name"])["percentile"]
    
    # Calculate theta
    step = (thrashold - abs(float(value["percentile"]) - float(BHV_M1_score)))
    
    if step > 0:
      step = 1
    else:
      step = 0
      
    
    BHV_M1_temp[value["name"]] = BHV_M1_score + ego * (value["percentile"] - BHV_M1_score) * step
  
  final_result_t1.append(BHV_M1_temp)  

# Calculate BCM for t2

In [0]:
final_result_t2 = []
index = 0
for j in valuesT2[1:4]:
  BHV_M2_temp = {}
  for value in j:
    BHV_M2_score = getPercentile(BHV_M2, value["name"])["percentile"]
    
    # calculate theta
    step = (thrashold - abs(float(value["percentile"]) - float(BHV_M2_score)))
    
    if step > 0:
      step = 1
    else:
      step = 0
    
    BHV_M2_temp[value["name"]] = BHV_M2_score + ego * (value["percentile"] - BHV_M2_score) * step
  
  final_result_t2.append(BHV_M2_temp)
  


In [0]:
print(json.dumps(BHV_M1, indent=2))
print(json.dumps(BHV_M2, indent=2))
print(json.dumps(final_result_t1, indent=2))
print(json.dumps(final_result_t2, indent=2))